In [ ]:
from pathlib import Path
import json
import pandas as pd
from uau_api.settings import Settings
from IPython.display import display, HTML
from requests.compat import urlparse, unquote
from uau_api import UauAPI
from uau_api.utils import write_jsonl

In [ ]:
wd = Path(Settings().WORKDIR)
uau = UauAPI(Settings().API_URL, Settings().API_KEY)
uau.authenticate('leonardo', 'hybr01')
spes = pd.read_excel(wd /'NotasUAU/SPEs_Cadastro.xlsx')
obras_ativas = uau.Obras.obter_obras_ativas()
obras_ativas = pd.DataFrame(obras_ativas)
# obras_ativas = obras_ativas[obras_ativas['Empresa_obr'].isin(spes['Código'])]

In [ ]:
with open(wd / 'NotasUAU/processos_pagamento.jsonl') as fc:
    processos_pagamento = [json.loads(line) for line in fc.readlines()]

In [ ]:
processos_pagamento = pd.json_normalize(processos_pagamento)

In [ ]:
obras_ativas = obras_ativas[obras_ativas['Empresa_obr'] >= 254]
obras_ativas = obras_ativas[obras_ativas['Empresa_obr'] <= 500]

In [ ]:
obras_ativas = obras_ativas[
    ~(obras_ativas['Empresa_obr'].isin(processos_pagamento['Empresa']))
    & ~(obras_ativas['Cod_obr'].isin(processos_pagamento['Obra']))
]

In [ ]:
obras_ativas[obras_ativas['Descr_obr'].str.contains('UFV')]

In [ ]:
for _, row in obras_ativas[obras_ativas['Descr_obr'].str.contains('UFV')].iterrows():
    empresa_obra_periodo = {
        'EmpresaObra': [{'Empresa': row['Empresa_obr'], 'Obra': row['Cod_obr']}],
        'PeriodoInicial': '2003-04-01T00:00:00.300Z',
        'PeriodoFinal': '2025-05-20T00:00:00.300Z',
    }
    response = uau.ProcessoPagamento.consultar_processos(
        empresa_obra_periodo=empresa_obra_periodo
    )
    if response:
        write_jsonl(response, wd / 'NotasUAU/processos_pagamento.jsonl', mode='a')

In [ ]:
processos_pagamento[0]['Parcelas'][0].keys()

In [ ]:
with open(wd / 'NotasUAU/downloaded.jsonl') as fc:
    file_contents = [json.loads(line) for line in fc.readlines()]

In [ ]:
df = pd.json_normalize(file_contents)

In [ ]:
def get_file_id(string):
    query = urlparse(string).query
    result = unquote(query) #273-420A-2-2024-04-10-00289348000736-63023-0.jpg
    return result.split('&', 1)[0].rsplit('/')[-1].split('.')[0]

In [ ]:
html = df.iloc[:10][['Empresa', 'Obra', 'to_download.Processo', 'url']].to_html(
    formatters={'url': lambda x: f'<a href="{x}">{get_file_id(x)}</a>'},
    escape=False
)
display(HTML(html))